In [2]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
from openmm import app
import nglview as nv
import mdtraj as md
import numpy as np
import math
import random

%load_ext autoreload
%autoreload 2

In [39]:
pdb = PDBFile('input.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-142841.16737778124,292.4823346014168
2000,-140684.86098129686,307.20571623618815
3000,-140991.63124496874,303.67959851635305
4000,-140873.8748973125,301.3297749155882
5000,-140423.5291941875,302.39453748521254
6000,-140199.4452098125,305.1229846055888
7000,-140511.44472153124,297.84233011693414
8000,-140590.797748875,304.47338765448916
9000,-141142.10194809374,298.02642694313795
10000,-140431.78358871874,299.90973318682455


In [5]:
# Load the trajectory and the topology (e.g., from your output PDB)
traj = md.load('train_trajectories/AAAAAGAGGAAAGGGAAAGGGAGAGAAGAAGAGAG.pdb', top='train_trajectories/AAAAAGAGGAAAGGGAAAGGGAGAGAAGAAGAGAG.pdb')

# Create an interactive widget to visualize the trajectory
view = nv.show_mdtraj(traj)
view

NGLWidget(max_frame=999)

In [13]:
residue_types = ["ALA", "GLY"]
sequence_length = 50

# Generate a sequence of 100 residues chosen at random.
sequence = [random.choice(residue_types) for _ in range(sequence_length)]

topology = Topology()
chain = topology.addChain()
positions = []
# Also store a numeric value for each residue type: 0.0 for ALA, 1.0 for GLY.
residue_numeric = []

for i, res_name in enumerate(sequence):
    residue = topology.addResidue(res_name, chain)
    # Add a single atom (using a carbon as a placeholder for the Cα).
    topology.addAtom("CA", Element.getByAtomicNumber(6), residue)
    
    # Assign a numeric type for later use in the force.
    res_type_num = 0.0 if res_name == "ALA" else 1.0
    residue_numeric.append(res_type_num)
    
    # Place atoms in an extended conformation along the x-axis with slight random noise.
    noise = Vec3(random.uniform(-0.1, 0.1),
                 random.uniform(-0.1, 0.1),
                 random.uniform(-0.1, 0.1)) * nanometer
    positions.append(Vec3(i * 0.38, 0, 0) * nanometer + noise)

md_topology = md.Topology.from_openmm(topology)

# Convert positions to numpy array in nanometers: shape (1, n_atoms, 3)
xyz = np.array([[pos.x, pos.y, pos.z] for pos in positions])[None, :]  # units already in nm

# Create MDTraj trajectory
traj = md.Trajectory(xyz, md_topology)

# Create and display NGLView widget
nv.show_mdtraj(traj)

NGLWidget()

In [10]:
from openmm.app import Topology, PDBReporter, StateDataReporter, Simulation, element
from openmm import System, CustomBondForce, CustomNonbondedForce, LangevinIntegrator, Vec3
from openmm.unit import *
import numpy as np
from sys import stdout

# ----- Parameters for our simple protein model -----
n_residues = 50
mass = 12.0             # mass (in daltons) for each bead (arbitrary choice)
bond_length = 0.38*nanometer   # typical Cα–Cα distance
bond_k = 1000.0 * kilojoule_per_mole / nanometer**2  # bond strength

# Lennard-Jones parameters (tune epsilon and sigma to adjust folding behavior)
epsilon = 0.5 * kilojoule_per_mole  # depth of the potential well
sigma = 0.3 * nanometer             # finite distance where potential is zero

# ----- Create a Topology for a linear chain -----
topology = Topology()
chain = topology.addChain()
positions = []  # will store initial positions for each bead

# Build the chain: one residue per bead, each with one atom (using carbon as a placeholder)
for i in range(n_residues):
    residue = topology.addResidue('RES', chain)
    # Use a generic element (here, carbon) for each bead.
    atom = topology.addAtom('C', element.carbon, residue)
    # Place beads in a straight line along the x-axis.
    positions.append(Vec3(i * bond_length.value_in_unit(nanometer), 0, 0) * nanometer)

# ----- Create the System and add particles -----
system = System()
for i in range(n_residues):
    system.addParticle(mass * dalton)

# ----- Bonded Interactions: Harmonic bonds between consecutive beads -----
bond_force = CustomBondForce("0.5 * k * (r - r0)^2")
bond_force.addPerBondParameter("k")
bond_force.addPerBondParameter("r0")
for i in range(n_residues - 1):
    bond_force.addBond(i, i+1, [bond_k, bond_length])
system.addForce(bond_force)

# ----- Nonbonded Interactions: Lennard-Jones potential between all pairs -----
# This potential can cause the chain to collapse/fold if attractions dominate.
nonbonded_force = CustomNonbondedForce("4*epsilon*((sigma/r)^12 - (sigma/r)^6)")
nonbonded_force.addGlobalParameter("epsilon", epsilon)
nonbonded_force.addGlobalParameter("sigma", sigma)
nonbonded_force.setCutoffDistance(1.5 * nanometer)
nonbonded_force.setNonbondedMethod(CustomNonbondedForce.CutoffNonPeriodic)

# Add each particle (no per-particle parameters in this simple example)
for i in range(n_residues):
    nonbonded_force.addParticle([])
system.addForce(nonbonded_force)

# ----- Set up the simulation -----
temperature = 300 * kelvin
integrator = LangevinIntegrator(temperature, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(topology, system, integrator)
simulation.context.setPositions(positions)

# Minimize energy to remove any bad contacts
simulation.minimizeEnergy()

# Reporters: Save snapshots to a PDB file and log simulation data to the console.
simulation.reporters.append(PDBReporter("folding_output.pdb", 100))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))

# ----- Run the Simulation -----
print("Starting simulation...")
simulation.step(5000)
print("Simulation complete. Check 'folding_output.pdb' for the trajectory.")

NGLWidget()

In [10]:
traj = md.load('folding_output.pdb')

# Create an interactive widget to visualize the trajectory
view = nv.show_mdtraj(traj)
view

NGLWidget(max_frame=49)

In [14]:
from openmm.app import PDBFile

pdb = PDBFile('input.pdb')
residues = list(pdb.topology.residues())
print("Number of residues:", len(residues))

Number of residues: 2798


In [77]:
from openmm.app import Topology, PDBFile, Element
from openmm.unit import nanometer
from openmm import Vec3
import numpy as np

# Define a simple sequence: 20 alanines.
sequence = ["ALA"] * 100

# Create the topology and positions.
topology = Topology()
chain = topology.addChain()
positions = []

for i, res_name in enumerate(sequence):
    residue = topology.addResidue(res_name, chain)
    # Use carbon as a placeholder element for the Cα atom.
    topology.addAtom("CA", Element.getByAtomicNumber(6), residue)
    # Place atoms in an extended conformation along the x-axis.
    noise = Vec3(random.uniform(-0.01,0.01),
                 random.uniform(-0.01,0.01),
                 random.uniform(-0.01,0.01)) * nanometer
    positions.append(Vec3(i * 0.38, 0, 0) * nanometer + noise)

# Convert positions to a NumPy array.
# This converts each Vec3 into a list of its x, y, z components.
positions_array = np.array([[p.x, p.y, p.z] for p in positions]) * nanometer

# Write out the PDB file.
with open("extended_chain.pdb", "w") as f:
    PDBFile.writeFile(topology, positions_array, f)

In [78]:
nv.show_file("extended_chain.pdb")

NGLWidget()

In [3]:
# --- Load the extended chain ---
pdb = PDBFile("extended_chain.pdb")
topology = pdb.topology
positions = pdb.getPositions()
n_residues = len(list(topology.residues()))
print("Number of residues:", n_residues)

# --- Create a coarse-grained CA-only system ---
system = System()
for i in range(n_residues):
    system.addParticle(12.0 * dalton)

# --- Force field parameters ---
# Bond: target distance ~0.38 nm
r0 = 0.38 * nanometer  
k_bond = 1000 * kilojoule_per_mole / nanometer**2

# Angle: target angle ~105° (≈1.83 rad)
angle0 = 1.83 * radian     
k_angle = 100 * kilojoule_per_mole / radian**2

# Torsion: bias toward an equilibrium dihedral of ~100° (≈1.745 rad)
dihedral0 = 1.745 * radian  
k_dihedral = 1 * kilojoule_per_mole  # increased further

# Nonbonded: full Lennard-Jones potential (with a modest epsilon)
# Using a smaller epsilon so the attraction doesn't over-collapse the chain.
epsilon_nb = 5 * kilojoule_per_mole  
sigma_nb = 0.3 * nanometer

# --- Bonded Forces ---
# Bond force between adjacent CA atoms.
bond_force = CustomBondForce("0.5 * k_bond * (r - r0)^2")
bond_force.addPerBondParameter("k_bond")
bond_force.addPerBondParameter("r0")
for i in range(n_residues - 1):
    bond_force.addBond(i, i+1, [k_bond, r0])
system.addForce(bond_force)

# Angle force for consecutive triplets.
angle_force = CustomAngleForce("0.5 * k_angle * (theta - angle0)^2")
angle_force.addPerAngleParameter("k_angle")
angle_force.addPerAngleParameter("angle0")
for i in range(n_residues - 2):
    angle_force.addAngle(i, i+1, i+2, [k_angle, angle0])
system.addForce(angle_force)

# --- Torsion Force ---
# Using the cosine form from the docs:
# Bias the dihedral angle toward theta0 with:
# "0.5 * k * (1 - cos(theta - theta0))"
torsion_force = CustomTorsionForce("0.5 * k * (1 - cos(theta - theta0))")
torsion_force.addPerTorsionParameter("k")
torsion_force.addPerTorsionParameter("theta0")
for i in range(n_residues - 3):
    torsion_force.addTorsion(i, i+1, i+2, i+3, [k_dihedral, dihedral0])
system.addForce(torsion_force)

# Now add an attractive contact potential between non-adjacent residues.
# This is just one possible form.
contact_force = CustomNonbondedForce(
    "step(r - rmin)*step(rmax - r)*(-epsilon_contact*exp(-((r - r_contact)^2)/(delta^2)))"
)
contact_force.addGlobalParameter("epsilon_contact", 0.5 * kilojoule_per_mole)
contact_force.addGlobalParameter("r_contact", 0.7 * nanometer)
contact_force.addGlobalParameter("delta", 0.1 * nanometer)
contact_force.addGlobalParameter("rmin", 0.5 * nanometer)  # only consider contacts beyond bonded neighbors
contact_force.addGlobalParameter("rmax", 1.5 * nanometer)
# Exclude interactions between directly bonded atoms:
contact_force.setNonbondedMethod(CustomNonbondedForce.NoCutoff)
for i in range(n_residues):
    contact_force.addParticle([])

# You might also want to exclude interactions between adjacent residues:
contact_force.addExclusion(0, 1)

# --- Nonbonded Force ---
# Now using the full Lennard-Jones potential (attractive and repulsive parts)
nonbonded_force = CustomNonbondedForce("4*epsilon*((sigma/r)^12 - (sigma/r)^6)")
nonbonded_force.addGlobalParameter("epsilon", epsilon_nb)
nonbonded_force.addGlobalParameter("sigma", sigma_nb)
nonbonded_force.setCutoffDistance(1.0 * nanometer)
nonbonded_force.setNonbondedMethod(CustomNonbondedForce.CutoffNonPeriodic)
for i in range(n_residues):
    nonbonded_force.addParticle([])
system.addForce(nonbonded_force)

# --- Set up the Simulation with Simulated Annealing ---
initial_temp = 300 * kelvin
final_temp = 100 * kelvin
n_intervals = 20
steps_per_interval = 2500  # Total steps: 20*250 = 5000

integrator = LangevinIntegrator(initial_temp, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(topology, system, integrator)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()

# Reporters: Save a PDB snapshot every 100 steps and log simulation data.
simulation.reporters.append(PDBReporter("folded_output.pdb", 10))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))

print("Starting simulated annealing folding simulation with enhanced forces...")
temps = np.linspace(initial_temp.value_in_unit(kelvin), final_temp.value_in_unit(kelvin), n_intervals) * kelvin
for T in temps:
    integrator.setTemperature(T)
    simulation.step(steps_per_interval)
    print("Temperature set to:", T)

print("Simulation complete. Check 'folded_output.pdb' for the trajectory.")

Number of residues: 100
Starting simulated annealing folding simulation with enhanced forces...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-154.11696553230286,260.6183212016583
2000,-92.05417823791504,276.73050366800663
Temperature set to: 300.0 K
3000,-139.6510558128357,259.6148225094881
4000,-115.19524663686752,278.65609593278526
5000,-187.36755800247192,243.29079173362138
Temperature set to: 289.4736842105263 K
6000,-264.29465889930725,313.03791619608427
7000,-333.0412087440491,275.84704600325324
Temperature set to: 278.9473684210526 K
8000,-412.40829133987427,311.5350519503479
9000,-382.43963503837585,271.82985174639646
10000,-433.75444078445435,308.31369941023814
Temperature set to: 268.42105263157896 K
11000,-479.88890540599823,259.6770918587446
12000,-467.6521165370941,240.3262086931979
Temperature set to: 257.89473684210526 K
13000,-525.1998500823975,225.88733761131923
14000,-531.7690300941467,270.1767048432861
15000,-554.3478760719299,218.71068139157575
Temper

In [1]:
import mdtraj as md
import nglview as nv

# Load the trajectory from your PDB file.
traj = md.load('trajectories/GGAAAAGAAAGAAGGGAAAG.pdb')

# Create the NGLView widget.
view = nv.show_mdtraj(traj)

# Clear default representations.
#view.clear_representations()
view.add_spacefill(selection="ALA", color="red", radius=0.5)
view.add_spacefill(selection="GLY", color="blue", radius=0.5)
# Add a cartoon representation for each residue type with a custom color.
#view.add_cartoon(selection="ALA", color="red")
#view.add_cartoon(selection="GLY", color="blue")

# Display the view.
view

NGLWidget(max_frame=999)

In [4]:
from ProteinGenerator import *
import numpy as np

nr_of_proteins = 99
generate_multiple_proteins(np.random.randint(15, 50, size=nr_of_proteins))

Starting simulated annealing folding simulation with enhanced forces...
Simulation complete. Check trajectories\GAGGAGAGAGAGGGGGAGGGAGGAGGAAAAAGGGAAAAGAG.pdb for the trajectory.
Starting simulated annealing folding simulation with enhanced forces...
Simulation complete. Check trajectories\GAAGGGAAGGGGAAAAAGGAGAGGGGGAAAAAAAA.pdb for the trajectory.
Starting simulated annealing folding simulation with enhanced forces...
Simulation complete. Check trajectories\AAGAGAGAAAGAAAGAAAAGAAAAAGGGGGAAGGAGAGA.pdb for the trajectory.
Starting simulated annealing folding simulation with enhanced forces...
Simulation complete. Check trajectories\AAGAGGGGAGGAAGGGAAAAGAGAGGGGAGGAGAAGA.pdb for the trajectory.
Starting simulated annealing folding simulation with enhanced forces...
Simulation complete. Check trajectories\AGAGAGGAGGAAGGAGGAAGGAAGGGAAGAAGGAGAG.pdb for the trajectory.
Starting simulated annealing folding simulation with enhanced forces...
Simulation complete. Check trajectories\AAGAAAAGGGAAAA

In [5]:
%load_ext autoreload
%autoreload 2
from ProteinTrajectoryDataset import *

dataset = ProteinTrajectoryDataset('trajectories/AAAAAGAAAGGGGGAGAGGAGGAGGAG.pdb', n_steps=10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading trajectories: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]


In [9]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [10]:
for batch in dataloader:
    current = batch['current']  # Shape: (batch_size, n_atoms, 3)
    delta = batch['delta']      # Shape: (batch_size, n_atoms, 3)
    print("Current shape:", current.shape)
    print("Delta shape:", delta.shape)
    # Insert your training code here.

Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shape: torch.Size([16, 27, 3])
Delta shape: torch.Size([16, 27, 3])
Current shap